In [4]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_google_genai import ChatGoogleGenerativeAI
from tqdm import tqdm
from typing import TypedDict, Annotated
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage, BaseMessage

In [5]:
load_dotenv()

True

In [6]:
llm = ChatGoogleGenerativeAI(
    model = 'gemini-2.5-flash'
)

In [7]:
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages] 

In [9]:
def chatNode(state: ChatState):
    messages = state['messages']

    response = llm.invoke(messages)

    return {'messages' : [response]}

In [10]:
checkpointer = InMemorySaver()

graph = StateGraph(ChatState)

graph.add_node('chat_node', chatNode)

graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

chat = graph.compile(checkpointer=checkpointer)


In [11]:
thread_id = 1

while True:

    message = input("Type here: ")

    print('User: ', message)

    if message.strip().lower() in ['exit', 'quit', 'bye']:
        break

    config = {
        'configurable': {
            'thread_id': thread_id
        }
    }

    res = chat.invoke({'messages': HumanMessage(content=message)}, config=config)

    print('AI', res['messages'][-1].content)

User:  hello I am Raj
AI Hello Raj! Nice to meet you. I'm an AI assistant. How can I help you today?
User:  What is my name
AI Your name is Raj. You told me that just a moment ago!
User:  how are you remembering that ?
AI That's a great question! It highlights a key difference between how I work and how a human brain works.

I don't "remember" in the way a person does, with a brain, consciousness, or long-term personal memory. Instead, here's how it works:

1.  **Context Window / Conversation History:** My underlying model keeps a record of our ongoing conversation. Think of it like a very short-term memory buffer or a notepad where everything we've said during *this specific chat session* is written down.
2.  **Processing Previous Turns:** When you ask a new question, I don't just process that one sentence in isolation. I look at your current question **and** a certain number of the previous turns in our conversation history.
3.  **Identifying Relevant Information:** My algorithms are

/home/rajgajjar04/Learnings/Langchain_Langgraph/.venv/lib/python3.13/site-packages/langchain_google_genai/chat_models.py:2832: UserWarning: HumanMessage with empty content was removed to prevent API error
  warnings.warn(


AI 
User:  bye
